In [1]:
%pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

import spacy
from spacy.cli.download import download as spacy_download

from azureml.core import Workspace, Dataset

In [3]:
subscription_id = '<<subscription_id>>'
resource_group = 'rg-leore-001'
workspace_name = 'leomathesisML'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='db_data_asset')
df = dataset.to_pandas_dataframe()

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Message: rslex failed, falling back to clex.
Payload: {"pid": 10443, "source": "azureml.dataprep", "version": "4.8.4", "trace": "azureml|data|tabular_dataset.py, line 169 in function <lambda>.\nazureml|data|dataset_error_handling.py, line 106 in function _try_execute.\nazureml|data|tabular_dataset.py, line 169 in function to_pandas_dataframe.", "subscription": "", "run_id": "", "resource_group": "", "w

In [4]:
spacy_model = 'de_core_news_lg'
spacy_download(spacy_model)
nlp = spacy.load(spacy_model)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 955.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [5]:
df = df[df['text'].isnull() == False]

In [6]:
df['iob_ident'] = df['session_id'].astype(str) + '_' + df['item_of_business']

In [7]:
df['lemma'] = None
df.reset_index(inplace=True)

In [8]:
nlp_pipe = nlp.pipe(df['text'], 
                    disable=['morphologizer', 'parser', 'senter', 'ner'],
                    n_process=-1)
for idx, doc in enumerate(nlp_pipe):
    df.at[idx, 'lemma'] = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space and not token.is_digit]

In [9]:
df.to_pickle("../data/df_lemma.pkl")